In [ ]:
import pandas as pd
import numpy as np
import joblib

# Models
import keras
from keras.models import Sequential
from keras.layers import Dense

#Tuning and Cross Validation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#### Importing Data

In [ ]:
x_train = pd.read_csv('../../Data Files/Training_Data/x_train.csv')
x_test = pd.read_csv('../../Data Files/Training_Data/x_test.csv')
y_train = pd.read_csv('../../Data Files/Training_Data/y_train.csv')
y_test = pd.read_csv('../../Data Files/Training_Data/y_test.csv')

#### Defining Neural Network Architecture

In [ ]:
def build_nn_model(num_neurons_1, num_neurons_2, activation_fn, optimizer_fn):
    nn = Sequential()
    nn.add(Dense(num_neurons_1, input_shape=[len(x_train.columns),], activation=activation_fn))
    nn.add(Dense(num_neurons_2, input_shape=[len(x_train.columns),], activation=activation_fn))
    nn.add(Dense(1, activation='linear'))
    nn.compile(loss='mean_squared_error', optimizer=optimizer_fn, metrics=['mean_absolute_error']) 
    return nn

#### Training Neural Network

In [ ]:
nn = build_nn_model(30, 30, 'relu', 'adam')
nn_history = nn.fit(x_train, y_train, batch_size=5, epochs=5, verbose=1)
y_pred_nn = nn.predict(x_test)

#### Tuning for Hyperparameters with Cross Validation

In [ ]:
model = KerasRegressor(build_fn=build_nn_model)

param_grid = {'num_neurons_1': [30, 50],
              'num_neurons_2': [10, 30, 50],
              'activation_fn': ['tanh', 'softplus', 'relu'],
              'optimizer_fn': ['adam', 'sgd'],
              'batch_size': [16, 64, 128],
              'epochs': [5, 10]}

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=skf, verbose=3)
grid_search.fit(x_train, y_train, verbose=0)

In [ ]:
optimal_clf = grid_search.best_estimator_

y_pred = optimal_clf.predict(x_test)
mae_after_tuning = mean_absolute_error(y_test, y_pred)
print("mean_absolute_error = {:.3}".format(mae_after_tuning)) 
print(grid_search.best_params_)

#### Generating Predictions

In [ ]:
tuned_nn = build_nn_model(30, 30, 'relu', 'adam')
nn_history = tuned_nn.fit(x_train, y_train, batch_size=5, epochs=5, verbose=1)
y_pred_nn = tuned_nn.predict(x_test)

#### Saving Model File and Predictions

In [ ]:
save_path = '../../Data Files/Model Files/'
nn.save(save_path + 'nn.h5')
joblib.dump(y_pred_nn, save_path + 'neuralnetwork_output.csv')